In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# import libraries

In [ ]:
%cd /content/drive/MyDrive/iti/NLP/project

/content/drive/MyDrive/iti/NLP/project


In [ ]:
! pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


# read preprocessed file

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/iti/NLP/project/preprocessed files/preprocessed_file_ML.csv", index_col=0)

In [ ]:
df

,text,dialect
id,,
1009754958479151232,قليلين ادب ومنافقين اختهم او قريبتهم تتعاكس تق...,LY
1009794751548313600,الليبيين متقلبين بالنسبة ليا انا ميليشياوي زما...,LY
1019989115490787200,تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها...,LY
1035479791758135168,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل...,LY
1035481122921164800,شكلك متعقدة علشان الراجل تحبيه ازوج بنت يتيمة ...,LY
...,...,...
982775354405376000,الناس دي بتنفخ قربة مقدودة بالدارجي كده البلد ...,SD
980728852224860160,انت عايش وين بره السودان شنو ماشايف البحصل دا,SD
975502734668500992,مااحرم نفسي ميسي حريف ولعاب برضو مدريدي وافتخر,SD


In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df.dropna(axis = 0, inplace = True)
df.isnull().sum().sum()

0

# machine learning model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.dialect, test_size=.2, stratify=df.dialect, random_state=42)

In [ ]:
vec = CountVectorizer()
clf = LinearSVC()
pipe = make_pipeline(vec, clf)
pipe.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('linearsvc', LinearSVC())])

In [ ]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

              precision    recall  f1-score   support

          EG       0.84      0.88      0.86     11518
          LB       0.84      0.82      0.83      5519
          LY       0.78      0.81      0.79      7296
          MA       0.80      0.68      0.73      2305
          SD       0.72      0.61      0.66      2883

    accuracy                           0.81     29521
   macro avg       0.79      0.76      0.78     29521
weighted avg       0.81      0.81      0.81     29521

accuracy: 0.810


## Enhancement

In [ ]:
# words
ngram_range=[1,1,2,3,5]

for i,j in zip(ngram_range,ngram_range[1:]):
    print(f"For N-Gram range = ({i,j})")
    vec = TfidfVectorizer( ngram_range=(i, j))
    clf_svc = LinearSVC()
    pipe_tfidf = make_pipeline(vec, clf_svc)
    pipe_tfidf.fit(x_train, y_train)
    print_report(pipe_tfidf, x_test , y_test)    
    print('#'*60)
    print('\n'*2)

For N-Gram range = ((1, 1))
              precision    recall  f1-score   support

          EG       0.84      0.92      0.88     11518
          LB       0.86      0.84      0.85      5519
          LY       0.81      0.82      0.81      7296
          MA       0.85      0.69      0.76      2305
          SD       0.80      0.61      0.69      2883

    accuracy                           0.83     29521
   macro avg       0.83      0.78      0.80     29521
weighted avg       0.83      0.83      0.83     29521

accuracy: 0.833
############################################################



For N-Gram range = ((1, 2))
              precision    recall  f1-score   support

          EG       0.83      0.93      0.88     11518
          LB       0.87      0.85      0.86      5519
          LY       0.82      0.82      0.82      7296
          MA       0.87      0.68      0.76      2305
          SD       0.82      0.60      0.69      2883

    accuracy                           0.84     2

In [ ]:
# char
ngram_range=[1,1,2,3,5]

for i,j in zip(ngram_range,ngram_range[1:]):
    print(f"For N-Gram range = ({i,j})")
    vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(i, j), min_df=.001, max_df=.3)
    clf_svc = LinearSVC()
    pipe_tfidf = make_pipeline(vec, clf_svc)
    pipe_tfidf.fit(x_train, y_train)
    print_report(pipe_tfidf, x_test , y_test)    
    print('#'*60)
    print('\n'*2)

For N-Gram range = ((1, 1))
              precision    recall  f1-score   support

          EG       0.42      0.89      0.57     11518
          LB       0.46      0.04      0.08      5519
          LY       0.32      0.19      0.23      7296
          MA       0.47      0.04      0.08      2305
          SD       0.40      0.00      0.00      2883

    accuracy                           0.41     29521
   macro avg       0.41      0.23      0.19     29521
weighted avg       0.40      0.41      0.30     29521

accuracy: 0.405
############################################################



For N-Gram range = ((1, 2))
              precision    recall  f1-score   support

          EG       0.69      0.81      0.75     11518
          LB       0.65      0.67      0.66      5519
          LY       0.58      0.63      0.61      7296
          MA       0.68      0.38      0.49      2305
          SD       0.62      0.28      0.39      2883

    accuracy                           0.65     2

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 2))
clf = LinearSVC()
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('linearsvc', LinearSVC())])

In [ ]:
print_report(pipe_tfidf, x_test, y_test)

              precision    recall  f1-score   support

          EG       0.83      0.93      0.88     11518
          LB       0.87      0.85      0.86      5519
          LY       0.82      0.82      0.82      7296
          MA       0.87      0.68      0.76      2305
          SD       0.82      0.60      0.69      2883

    accuracy                           0.84     29521
   macro avg       0.84      0.78      0.80     29521
weighted avg       0.84      0.84      0.83     29521

accuracy: 0.838


In [ ]:
import joblib
joblib.dump(pipe_tfidf, '/content/drive/MyDrive/iti/NLP/project/models/tfidf_pipeline.pkl')

['/content/drive/MyDrive/iti/NLP/project/tfidf_pipeline.pkl']

# testing

In [ ]:
mapping = {"EG":"Egypt", "LB": "Lebanon", "LY": "Libya", "MA": "Morocco", "SD":"Sindhi"}
text = [" نتي زوينة"]
text = [preprocessing.processing(x) for x in text]
print(text)
y_pred = pipe_tfidf.predict(text)
for i in range(len(text)):
  print(f'{text[i]} --> {mapping[y_pred[i]]}')


['نتي زوينة']
نتي زوينة --> Morocco
